# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv("Output/city_data.csv")
city_weather_df

,Unnamed: 0,City,Country,Latitude,Longitude,Date,Temperature (F),Temp Max,Humidity (%),Cloudiness (%),Windspeed (mph)
0,0,Jennings,US,38.72,-90.26,1600208260,78.69,81.00,44,75,5.82
1,1,Bluff,NZ,-46.60,168.33,1600208086,48.99,48.99,79,100,14.00
2,2,Kapaa,US,22.08,-159.32,1600208097,84.79,86.00,58,20,10.29
3,3,Lebu,CL,-37.62,-73.65,1600208260,50.95,50.95,96,100,6.06
4,4,Grand-Santi,GF,4.25,-54.38,1600208260,75.85,75.85,90,81,0.54
...,...,...,...,...,...,...,...,...,...,...,...
579,579,Selaphum,TH,16.03,103.94,1600208316,77.11,77.11,85,77,2.42
580,580,Turochak,RU,52.26,87.12,1600208316,47.59,47.59,80,82,3.53
581,581,Tingi,TZ,-11.30,35.03,1600208316,68.38,68.38,52,31,6.89
582,582,Nelson Bay,AU,-32.72,152.15,1600207872,59.63,60.01,93,20,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_weather_df[["Latitude", "Longitude"]].astype(float)

# Create a poverty Heatmap layer
humidity_fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=city_weather_df['Humidity (%)'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

humidity_fig.add_layer(heat_layer)
humidity_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_cities_df = city_weather_df.loc[(city_weather_df["Temp Max"] > 75) & (city_weather_df["Temp Max"] < 90) & (city_weather_df["Cloudiness (%)"] < 25), :]
ideal_cities_df = ideal_cities_df.dropna(how='any')
del ideal_cities_df['Unnamed: 0']
ideal_cities_df=ideal_cities_df.reset_index(drop=True)
ideal_cities_df

,City,Country,Latitude,Longitude,Date,Temperature (F),Temp Max,Humidity (%),Cloudiness (%),Windspeed (mph)
0,Kapaa,US,22.08,-159.32,1600208097,84.79,86.00,58,20,10.29
1,Atuona,PF,-9.80,-139.03,1600208090,78.39,78.39,76,16,18.70
2,Vilhena,BR,-12.74,-60.15,1600208215,89.60,89.60,29,15,8.05
3,George Town,MY,5.41,100.34,1600208114,77.63,78.80,94,20,2.24
4,Half Moon Bay,US,37.46,-122.43,1600208265,73.85,77.00,72,1,16.11
5,Fortuna,US,40.60,-124.16,1600208267,77.32,79.00,61,20,12.75
6,Santa Cruz de la Sierra,BO,-17.80,-63.17,1600208268,77.00,77.00,50,0,6.93
7,Puerto Ayora,EC,-0.74,-90.35,1600207833,73.26,78.80,57,20,23.04
8,Lantana,US,26.59,-80.05,1600208163,86.25,87.80,70,1,9.17
9,Bagar,IN,28.19,75.50,1600208273,84.99,84.99,40,0,5.39


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
